In [311]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [312]:
from lib import jsoncorpus, datastuff
import gensim
import traceback
import numpy as np
import matplotlib.pyplot as plt
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.svm
import sklearn.decomposition
from lib.scikitComponents import *
import itertools
import collections

# LDA

## Data preprocessing

In [447]:
# Load in the previously created corpus and dictionary of scraped sites
dictionary, corpus, meta_corpus, dmoz_data = jsoncorpus.load_or_create('docs/sites.jl')

Done.


In [446]:
X = dmoz_data["meta"]
lists = [set(x.get('keyphrases', [])) | set(x.get('keywords', [])) for x in X]
sanitize = lambda word: word.strip()
counts = collections.Counter(sanitize(word) for word in itertools.chain(*lists))
stopwords = ['','and','to','for','the','of']
featureset = [x for x in counts if counts[x] > 4 and x not in stopwords]
print len(featureset)
[[x,counts[x]] for x in counts if counts[x] < 5]


1068


[[u'irvine startup lawyer', 1],
 [u'maths formula', 1],
 [u'woods', 1],
 [u'get more leads', 1],
 [u'general reference', 1],
 [u'woody', 1],
 [u'san diego engineer attorney', 1],
 [u'sevens', 1],
 [u'sexual assault', 1],
 [u'branded formulations', 1],
 [u'atlanta technology lawyer', 1],
 [u'textron airland', 1],
 [u'us literary agents', 1],
 [u'orange county engineer law firm', 1],
 [u'wooden', 3],
 [u'silicon hills content licensing attorney', 1],
 [u'worksafebc', 1],
 [u'feasibility', 1],
 [u'gorman', 1],
 [u'reengineering', 1],
 [u'prosody', 1],
 [u'tango down', 1],
 [u'sacramento trademark lawyer', 1],
 [u'biotech licensing law firm', 1],
 [u'lmt', 1],
 [u'north carolina saas attorney', 1],
 [u'los angeles software licensing attorney', 1],
 [u'clint peterson', 1],
 [u"cookin'", 1],
 [u'natures', 2],
 [u'climber', 3],
 [u'golden', 1],
 [u'girlsfrag', 1],
 [u'projection', 2],
 [u'sacramento engineer law firm', 1],
 [u'wolf armouries', 1],
 [u'k8055 velleman macosx', 1],
 [u'san franc

In [341]:
X = dmoz_data["meta"]
lists = [set(x.get('keyphrases', [])) | set(x.get('keywords', [])) for x in X]
sanitize = lambda word: word.strip()
counts = collections.Counter(sanitize(word) for word in itertools.chain(*lists))
words = [x for x in counts if counts[x] > 4 and x != '']
features = np.zeros((len(X), len(words)), dtype=int)

for i, doc in enumerate(lists):
    features[i,[words.index(x) for x in doc if x in words]] += 1

In [448]:
# Build a list of all topics
allcategories = set(tuple(t) for t in dmoz_data['dmoz_categories'])
# Build a list of all top-level topics
topcategories = set(topic[0] for topic in dmoz_data['dmoz_categories'])
# Link topics to URLs
meta = list(zip(dmoz_data['urls'], dmoz_data['dmoz_categories']))
# Represent the topics in an alternative way
heirarchal_categories = lambda max_depth: [['; '.join(topics[:ti+1]) for ti, t in enumerate(topics) if ti < max_depth] for topics in dmoz_data['dmoz_categories']]
# Top categories
top_categories = [x[0] for x in heirarchal_categories(1)]

## Define a classification pipeline for the corpus data

The Pipeline() object chains together objects from the lib.scikitComponents file, so that they can be used as part of a scikit-learn classification pipeline

In [315]:
X = dmoz_data['meta']
pipe = lda_feature_pipeline(10)
pipe.fit(X)
pipe.transform(X)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Define a classifier (decision trees), and chain the preprocessing step to the classifier

In [454]:
# classifier = sklearn.svm.LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
#      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
#      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
#      verbose=0)

# classifier = sklearn.svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
# gamma=0.0, kernel='rbf', max_iter=-1, probability=False, random_state=None,
# shrinking=True, tol=0.001, verbose=False)

classifier = sklearn.ensemble.ExtraTreesClassifier(random_state=0, n_estimators=100, oob_score=True, bootstrap=True, n_jobs=4)

clf = sklearn.pipeline.Pipeline([
    ('matrix_builder', MetaMatrixBuilder()),
    ('classification', classifier)])


In [19]:
#matrix = lda_feature_pipeline(len(top_categories)).fit_transform(np.array(meta_corpus))

In [20]:
#pca = sklearn.decomposition.PCA(n_components=0.9)
#pca.fit_transform(np.array(matrix))
#[[dictionary[w] for w in np.where(np.abs(comp)>0.08)[0]] for comp in pca.components_]

## Process the dmoz categories
In this case, we will just turn the top category in to an index

In [450]:
dmoz_encoder = sklearn.preprocessing.LabelEncoder().fit(top_categories)
classes = dmoz_encoder.transform(top_categories)
print classes

[7 7 7 ..., 3 3 3]


## Run the classifier

We can use the meta corpus or the full body corpus here, just by replacing a single argument

The data is split in to training and test sets, and then fit to the training set. The LDA model is generated ***only*** from the training set, not the testing set.

In [455]:
scores = sklearn.cross_validation.cross_val_score(clf, X, classes, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.28 (+/- 0.04)


In [456]:
X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(np.array(X), classes)

In [457]:
model = clf.fit(X_train, y_train)
print model.score(X_test, y_test)
# model.predict(X_test)
# X_test[0].get("keyphrases",[])

0.283333333333


In [411]:
category_scores = collections.defaultdict(float)
category_counts = collections.defaultdict(float)
for real, pred in sorted(zip(dmoz_encoder.inverse_transform(y_test), dmoz_encoder.inverse_transform(model.predict(X_test)))):
    category_scores[real] += 1 if real == pred else 0
    category_counts[real] += 1 

### Find the score for each category. 

Note that if the classifier assigned *random* categories, the score would be $\frac{1}{\textrm{num categories}}$. Instead, it is actually quite high in some cases, indicating a moderate amount of success (given how naive this is!)

In [419]:
print "1/num_cats = ", float(1)/float(15), "\n"
print "Category       Proportion   Score"
print "---------------------------------"
for cat, proportion, score in [(k, category_counts[k]/len(y_test), category_scores[k] / category_counts[k]) for k in category_counts.keys()]:
    print "{:<15} {:<7.4}      {:<5.4}".format(cat, proportion, score)

1/num_cats =  0.0666666666667 

Category       Proportion   Score
---------------------------------
arts            0.07333      0.1591
regional        0.001667      0.0  
shopping        0.05833      0.1714
reference       0.07         0.1667
business        0.06833      0.1463
kids and teens  0.035        0.1905
computers       0.095        0.2105
recreation      0.08         0.08333
sports          0.16         0.9271
society         0.06833      0.2195
health          0.04333      0.2308
home            0.05833      0.2571
games           0.04167      0.2  
news            0.06333      0.2895
science         0.08333      0.22 
